In [1]:
import dcachefs
import geopandas

# Create Arctic DEM index for 10m- and 32m- resolution tiles

Tiles are available for various resolutions, but the index is only available for the highest resolution data (2m). We store the index with the URLs to the derived 10m- and 32m-resolution tiles on dCache in GeoJSON format.

In [2]:
index_url = ("http://data.pgc.umn.edu/elev/dem/setsm/"
             "ArcticDEM/indexes/ArcticDEM_Tile_Index_Rel7.zip")
token_path = "./macaroon.dat"
dcache_path = ("https://webdav.grid.surfsara.nl:2880"
               "/pnfs/grid.sara.nl/data/eratosthenes/"
               "disk/GIS/Elevation")

In [3]:
index = geopandas.read_file(index_url)
index.head()

,objectid,name,tile,nd_value,resolution,creationda,raster,fileurl,spec_type,qual,reg_src,num_gcps,meanresz,active,qc,rel_ver,num_comp,st_area_sh,st_length_,geometry
0,9227,38_18_2_1_2m_v3.0,38_18,-9999.0,2.0,2018-05-24,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,DEM,0.999471,ICESat,892,-0.005,1,2,7,48,2.500000e+09,200000.0,"POLYGON ((-2250000.000 -250000.000, -2200000.0..."
1,9228,38_18_1_2_2m_v3.0,38_18,-9999.0,2.0,2018-05-24,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,DEM,0.997790,ICESat,892,-0.005,1,2,7,48,2.500000e+09,200000.0,"POLYGON ((-2300000.000 -200000.000, -2250000.0..."
2,9226,38_18_1_1_2m_v3.0,38_18,-9999.0,2.0,2018-05-24,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,DEM,0.996557,ICESat,892,-0.005,1,2,7,48,2.500000e+09,200000.0,"POLYGON ((-2300000.000 -250000.000, -2250000.0..."
3,9225,38_18_2_2_2m_v3.0,38_18,-9999.0,2.0,2018-05-24,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,DEM,0.998219,ICESat,892,-0.005,1,2,7,48,2.500000e+09,200000.0,"POLYGON ((-2250000.000 -200000.000, -2200000.0..."
4,1926,47_07_2_2_2m_v3.0,47_07,-9999.0,2.0,2018-07-14,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,DEM,0.013685,None,0,0.000,1,2,7,4,2.500000e+09,200000.0,"POLYGON ((-3350000.000 700000.000, -3300000.00..."


In [4]:
with open('macaroon.dat') as f:
    macaroon = f.read().strip()

dcache = dcachefs.dCacheFileSystem(
    api_url="https://dcacheview.grid.surfsara.nl:22880/api/v1",
    token=macaroon
)

In [7]:
for res in (10, 32):
    print(f"Index for DEM at {res}m resolution")
    new_index = index[['tile', 'fileurl', 'geometry']]
    new_index = new_index.dissolve(by="tile")

    new_urls = new_index.fileurl.str.replace('/2m/', f'/{res}m/').str.replace('_\d_\d_2m', f'_{res}m')
    new_index['fileurl'] = new_urls

    print(new_index.head().fileurl.values)
    
    urlpath = (dcache_path 
               + f"/ArcticDEM_Tile_Index_Rel7_{res}m.geojson")
    with dcache.open(urlpath, "wb") as f:
        new_index.to_file(f, driver='GeoJSON')

Index for DEM at 10m resolution


<ipython-input-7-419f231580ce>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  new_urls = new_index.fileurl.str.replace('/2m/', f'/{res}m/').str.replace('_\d_\d_2m', f'_{res}m')


['http://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/mosaic/v3.0/10m/07_40/07_40_10m_v3.0.tar.gz'
 'http://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/mosaic/v3.0/10m/07_41/07_41_10m_v3.0.tar.gz'
 'http://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/mosaic/v3.0/10m/07_42/07_42_10m_v3.0.tar.gz'
 'http://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/mosaic/v3.0/10m/08_39/08_39_10m_v3.0.tar.gz'
 'http://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/mosaic/v3.0/10m/08_40/08_40_10m_v3.0.tar.gz']
Index for DEM at 32m resolution


<ipython-input-7-419f231580ce>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  new_urls = new_index.fileurl.str.replace('/2m/', f'/{res}m/').str.replace('_\d_\d_2m', f'_{res}m')


['http://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/mosaic/v3.0/32m/07_40/07_40_32m_v3.0.tar.gz'
 'http://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/mosaic/v3.0/32m/07_41/07_41_32m_v3.0.tar.gz'
 'http://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/mosaic/v3.0/32m/07_42/07_42_32m_v3.0.tar.gz'
 'http://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/mosaic/v3.0/32m/08_39/08_39_32m_v3.0.tar.gz'
 'http://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/mosaic/v3.0/32m/08_40/08_40_32m_v3.0.tar.gz']


In [8]:
# test reading it again
with dcache.open(urlpath) as f:
    index_tmp = geopandas.read_file(f)
    
index_tmp.head()

,tile,fileurl,geometry
0,07_40,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,"POLYGON ((-50000.000 -3300000.000, 0.000 -3300..."
1,07_41,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,"POLYGON ((100000.000 -3400000.000, 50000.000 -..."
2,07_42,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,"POLYGON ((100000.000 -3300000.000, 150000.000 ..."
3,08_39,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,"POLYGON ((-100000.000 -3300000.000, -150000.00..."
4,08_40,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,"POLYGON ((0.000 -3300000.000, -50000.000 -3300..."
